In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlcontext = SQLContext(sc)
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1592377355348_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
weighted=sqlcontext.read.parquet("hdfs://master:9000/user/hadoop/weighted_vector")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
weighted.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+--------------------+
|token|title_id|            weighted|
+-----+--------+--------------------+
| 가따|   64997|[-0.0770580247044...|
| 가따|   72497|[-0.0770580247044...|
| 가따|  626907|[-0.0770580247044...|
| 가따|  551647|[-0.0770580247044...|
| 가따|  702608|[-0.0770580247044...|
| 가따|  710754|[-0.0770580247044...|
| 가따|  471283|[-0.0770580247044...|
| 가따|  524520|[-0.0770580247044...|
| 가따|  670144|[-0.0770580247044...|
| 가따|  709732|[-0.0770580247044...|
| 가따|  710768|[-0.0770580247044...|
| 가따|  727189|[-0.0770580247044...|
| 가따|  683496|[-0.0770580247044...|
| 가따|  563786|[-0.0770580247044...|
| 가렌|   64997|[-0.0327281914651...|
| 가렌|  602287|[-0.0327281914651...|
| 가렌|  602910|[-0.2290973402559...|
| 가렌|  551647|[-0.3927382975816...|
| 가렌|  670152|[-0.0327281914651...|
| 가렌|  730425|[-0.0327281914651...|
+-----+--------+--------------------+
only showing top 20 rows

In [9]:
token_count=sqlcontext.read.parquet("hdfs://master:9000/user/hadoop/token_count")
token_count.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+-----+
|title_id| token|count|
+--------+------+-----+
|  635187|  늑대|  113|
|  635187|작가님|   84|
|  635187|  진짜|   75|
|  635187|  하는|   69|
|  635187|  보고|   65|
|  635187|  벌레|   65|
|  635187|  으로|   61|
|  635187|  에서|   57|
|  635187|  빨간|   53|
|  635187|  사람|   52|
|  635187|  웹툰|   46|
|  635187|  생각|   44|
|  635187|  라고|   42|
|  635187|합니다|   42|
|  635187|  하고|   41|
|  635187|  있는|   40|
|  635187|  정말|   40|
|  635187|  연의|   39|
|  635187|  만화|   39|
|  635187|  프로|   38|
+--------+------+-----+
only showing top 20 rows

In [11]:
token_sum = token_count.groupBy('title_id').sum('count')
token_sum.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+
|title_id|sum(count)|
+--------+----------+
|   25517|        43|
|  400735|      2171|
|  721109|     94829|
|  738145|     10156|
|  671421|     33787|
|  737628|      7343|
|  739113|      3042|
|  662774|    284145|
|  703841|     56724|
|  670131|     25523|
|  729044|     11453|
|  737836|      7117|
|  719508|     13616|
|  739292|     19523|
|  703846|    151620|
|  317362|      7300|
|  721260|     28976|
|  498158|     10977|
|  729049|     25717|
|  374970|     11049|
+--------+----------+
only showing top 20 rows

In [14]:
vector_agg=F.udf(lambda list_of_vec, total_tf: sum(list(list_of_vec))/total_tf,VectorUDT())
doc2vec = weighted.join(token_sum, 'title_id').select('token','title_id','weighted', 'sum(count)').groupBy('title_id', 'sum(count)').agg(vector_agg(F.collect_list('weighted'),'sum(count)').alias('doc2vec'))
doc2vec

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[title_id: int, sum(count): bigint, doc2vec: vector]

In [15]:
doc2vec.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+--------------------+
|title_id|sum(count)|             doc2vec|
+--------+----------+--------------------+
|   25517|        43|[-0.0105203234503...|
|  400735|      2171|[-0.0027542388064...|
|  671421|     33787|[-0.0054135264087...|
|  721109|     94829|[0.00701903070860...|
|  738145|     10156|[-0.0208011262970...|
|  662774|    284145|[-0.0196802492258...|
|  670131|     25523|[-0.0128888864669...|
|  703841|     56724|[-5.9259301955923...|
|  737628|      7343|[-6.1573829728346...|
|  739113|      3042|[-0.0085619193076...|
|  719508|     13616|[-0.0097138986078...|
|  729044|     11453|[-0.0135098049821...|
|  737836|      7117|[-0.0137159900368...|
|  739292|     19523|[-0.0101766710384...|
|  317362|      7300|[0.00358121572659...|
|  498158|     10977|[-0.0081041887773...|
|  703846|    151620|[-0.0020181413243...|
|  721260|     28976|[-0.0058713119659...|
|  729049|     25717|[-0.0063999681697...|
|  318995|    515771|[-0.0199124875127...|
+--------+-

In [16]:
doc2vec.select('title_id','doc2vec').write.parquet("hdfs://master:9000/user/hadoop/doc2vec")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
d2v = sqlcontext.read.parquet("hdfs://master:9000/user/hadoop/doc2vec")

d2v

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[title_id: int, doc2vec: vector]

In [25]:
from pyspark.sql.types import *
d2v_csv=d2v.withColumn('d2v', d2v['doc2vec'].cast(StringType()))
d2v_csv.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+
|title_id|             doc2vec|                 d2v|
+--------+--------------------+--------------------+
|  186814|[-0.0020889922624...|[-0.0020889922624...|
|  563786|[-0.0183606380001...|[-0.0183606380001...|
|  680193|[-0.0040888356811...|[-0.0040888356811...|
|  685989|[-0.0042875406409...|[-0.0042875406409...|
|  703852|[-0.0090778826599...|[-0.0090778826599...|
+--------+--------------------+--------------------+
only showing top 5 rows

In [26]:
d2v_csv.select('title_id','d2v').write.csv("hdfs://master:9000/user/hadoop/doc2vec_csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…